In [1]:
## Performs automatic data conditioning steps on the Momentum Product Manual JSON data set extracted via the Web Scraper.
## Removes section and article numbering from the respective titles and replaces empty article bodies with a default message.
## Also applies manual adjustments created via modifications to the file generated by MPM_ManualAdjGenerator.py


#-------------- SETUP --------------
# Import required libraries
import os
import json
import re

In [2]:
# Set environment variables & settings (edit these to fit environment)
#Input File
input_file_name           = "MPM.json"
input_file_root           = ".."
input_file_folder         = os.path.join("Data", "Scraping")
#Output File
output_file_name          = "MPM_conditioned.json"
output_file_root          = ".."
output_file_folder        = os.path.join("Data", "Conditioning")
#JSON file containing manual article adjustments
manadj_file_name          = "MPM_manual_adj.json"
manadj_file_root          = ".."
manadj_file_folder        = os.path.join("Data", "Conditioning")
#Replacement message for empty article bodies
empty_article_replacement = "There is currently no information available on this topic. \n" +\
                            "Please contact Accure customer support for additional assistance."

# Set other variables
input_file_path           = os.path.join(input_file_root, input_file_folder, input_file_name)
output_file_path          = os.path.join(output_file_root, output_file_folder, output_file_name)
manadj_file_path          = os.path.join(manadj_file_root, manadj_file_folder, manadj_file_name)

In [3]:
#-------------- FUNCTIONS --------------
# Define function to remove the numbers from the section and article titles
def remove_title_numbers(article):
    article['Article_Title'] = re.sub("[\d\.]+\s*", '', article['Article_Title'])
    article['Section_Title'] = re.sub("[\d\.]+\s*", '', article['Section_Title'])
    
# Define function to replace the body for empty articles with a default replacement string
def replace_empty_articles(article):
    if not article['Article_Body']:
        article['Article_Body'] = empty_article_replacement
        
# Define function to override article bodies with manual adjustments where provided
def apply_manual_adjustments(article):
    replacement_body = [match['Article_Body'] for match in manadj if match['Product_Title'] == article['Product_Title']
                                                                 and match['Section_Title'] == article['Section_Title']
                                                                 and match['Article_Title'] == article['Article_Title']][0]
    if replacement_body:
        article['Article_Body'] = replacement_body
        
# Define function to strip border unicode, newline, and whitespace characters
def strip_border_unicode(article):
    article['Article_Body'] = re.sub("^(\\n|\\u00a0| )+", '', article['Article_Body'])
    article['Article_Body'] = re.sub("(\\n|\\u00a0| )+$", '', article['Article_Body'])
    
# Define function to replace unicode spaces with regular and trim all spaces down to a single one
def trim_spaces(article):
    # Replace all unicode spaces with regular spaces
    article['Article_Body'] = re.sub("\u00a0", ' ', article['Article_Body'])
    # Replace all instances of more than one whitespace character in a row with a single space
    article['Article_Body'] = re.sub(' {2,}', ' ', article['Article_Body'])

In [4]:
#-------------- MAIN --------------
# Import the contents of the MPM and manual adjustment JSON files
f = open(input_file_path, encoding="utf-8")
data = json.load(f)
f.close()

f = open(manadj_file_path, encoding="utf-8")
manadj = json.load(f)
f.close()

In [5]:
# Iterate through each JSON object and apply cleaning steps to them
for i in range(len(data)):
    apply_manual_adjustments(data[i])
    replace_empty_articles(data[i])
    remove_title_numbers(data[i])
    strip_border_unicode(data[i])
    trim_spaces(data[i])

In [6]:
# Create a new JSON file and write the cleaned product manual JSON objects to it
f = open(output_file_path, "w", encoding="utf-8")
f.write(json.dumps(data, indent=4))
f.close()